In [1]:
!pip install 'zarr<3'
!pip install timm
!pip install openslide-python tiffslide
!pip install cupy-cuda12x  # for CUDA 12.x

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached tiffslide-2.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached imagecodecs-2025.11.11-cp311-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (20 kB)
Using cached tiffslide-2.5.1-py3-none-any.whl (32 kB)
Using cached imagecodecs-2025.11.11-cp311-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (23.2 MB)

[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached cu

In [1]:
# ALWAYS RUN THIS FIRST!
import os
import sys
from pathlib import Path

NOTEBOOK_DIR = Path("/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest")
os.chdir(NOTEBOOK_DIR)
sys.path.insert(0, str(NOTEBOOK_DIR))

print(f"✅ Working directory: {os.getcwd()}")


✅ Working directory: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest


## WSI

In [2]:
import torch
from vitaminp import VitaminPFlex
from vitaminp.inference import ChannelConfig, WSIPredictor
# Setup model
device = 'cuda'
model = VitaminPFlex(model_size='large').to(device)
model.load_state_dict(torch.load("checkpoints/vitamin_p_flex_large_fold2_best.pth", map_location=device))
model.eval()

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ VitaminPFlex initialized with large backbone
  Architecture: Shared Encoder → 4 Separate Decoders
  Embed dim: 1024 | Decoder dims: [1024, 512, 256, 128]


VitaminPFlex(
  (backbone): DINOv2Backbone(
    (dinov2): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): M

## HE or mIF

In [7]:
from vitaminp.inference.channel_config import ChannelConfig
from vitaminp.inference import WSIPredictor

# Create channel config
config = ChannelConfig(
    nuclear_channel=0,  # SYTO 13
    membrane_channel=[1, 2],  # Combine all markers
    membrane_combination='max',
    channel_names={0: 'SYTO13', 1: 'Cy3', 2: 'TexasRed'}
)

# Create predictor with MIF config
predictor = WSIPredictor(
    model=model,
    device='cuda',
    patch_size=512,           # ← ADD THIS
    overlap=64,               # ← ADD THIS
    magnification=40,         # ← ADD THIS
    mif_channel_config=config # ← Your channel config
)

# Run MIF inference
results = predictor.predict(
    wsi_path='test_images/svs10.tif',
    output_dir='results',  # ← ADD THIS (where to save results)
    branch='he_nuclei',
    filter_tissue=True,
    tissue_threshold=0.1,      # Lower for MIF (darker images)
    clean_overlaps=True,        # ← ADD THIS (remove duplicates)
    save_geojson=True,
)

print(f"✅ Found {results['num_detections']} nuclei in {results['processing_time']:.2f}s")

2025-12-30 20:29:39 - WSIPredictor - INFO - WSIPredictor initialized:
2025-12-30 20:29:39 - WSIPredictor - INFO -   Device: cuda
2025-12-30 20:29:39 - WSIPredictor - INFO -   Patch size: 512
2025-12-30 20:29:39 - WSIPredictor - INFO -   Overlap: 64
2025-12-30 20:29:39 - WSIPredictor - INFO -   Magnification: 40
2025-12-30 20:29:39 - WSIPredictor - INFO -   MIF channels: Nuclear: SYTO13, Membrane: max(Cy3, TexasRed)
2025-12-30 20:29:39 - WSIPredictor - INFO - 📁 Opening WSI: test_images/svs10.tif
2025-12-30 20:29:39 - WSIPredictor - INFO -    ✓ Size: 3000x3000 pixels
2025-12-30 20:29:39 - WSIPredictor - INFO - 📐 Extracting tile positions...


⚠️  OpenSlide failed, trying TiffReader
   Scanning 7x7 tile grid...


2025-12-30 20:29:41 - WSIPredictor - INFO -    ✓ Created 49 tiles (7x7 grid)
2025-12-30 20:29:41 - WSIPredictor - INFO -    ✓ Tissue tiles: 49/49 (100.0%)
2025-12-30 20:29:41 - WSIPredictor - INFO - 🧠 Running predictions and extracting instances on he_nuclei...


   Tissue dilation: 47 → 49 tiles (+2 boundary tiles)


Processing tiles: 100%|██████████| 49/49 [00:13<00:00,  3.63it/s]
2025-12-30 20:29:54 - WSIPredictor - INFO -    ✓ Extracted 10046 instances from tiles (before cleaning)
2025-12-30 20:29:54 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2025-12-30 20:29:54 - WSIPredictor - INFO -    Found 2858 edge cells to check for overlaps
2025-12-30 20:29:54 - WSIPredictor - INFO - Starting overlap cleaning with 10046 detections
2025-12-30 20:29:54 - WSIPredictor - INFO - Split detections: 0 mid, 10046 edge
2025-12-30 20:29:55 - WSIPredictor - INFO - Iteration 1: Found 2766 overlaps, keeping 7280/10046 detections
2025-12-30 20:29:56 - WSIPredictor - INFO - Iteration 2: Found 42 overlaps, keeping 7238/7280 detections
2025-12-30 20:29:56 - WSIPredictor - INFO - Iteration 3: Found 0 overlaps, keeping 7238/7238 detections
2025-12-30 20:29:56 - WSIPredictor - INFO - No more overlaps found, stopping early
2025-12-30 20:29:56 - WSIPredictor - INFO - Cleaning complete: 10046

⚠️  OpenSlide failed, trying TiffReader


2025-12-30 20:30:00 - WSIPredictor - INFO - ✅ Complete! 7238 detections in 20.84s


✅ Found 7238 nuclei in 20.84s
